In [36]:
import building_tokenizer as toker
import json
import pandas as pd
from collections import Counter, OrderedDict
from copy import deepcopy
from functools import reduce
from IPython.display import Audio
import pdb

In [37]:
#naeListDict['Applied_Physics_and_Mathematics'] = ['514', '513', '604']
#naeListDict['Computer_Science_and_Engineering_Building'] = ['505', '506']
#naeListDict['Conrad_Prebys_Music_Center'] = ['523']
#naeListDict['San_Diego_Supercomputer_Center'] = ['668', '575', '524', '520']
#naeListDict['Literature_Building'] = ['553']
#naeListDict['Structural_and_Materials_Engineering_Building'] = ['572', '573', '574']
#naeListDict['Jacobs_Hall'] = ['620', '621', '622', '623', '650', '639', '638', '642', '643', '640', \
#                              '641', '637', '646', '644', '645']
#naeListDict['Warren_Lecture_Hall'] = ['555', '552']
#naeDict = dict()
#naeDict['bonner'] = ["607", "608", "609", "557", "610"]
#naeDict['ap_m'] = ['514', '513','604']
#naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
#naeDict['ebu3b'] = ["505", "506"]
#naeDict['music'] = ['523']
#naeDict['sme'] = ['572', '573', '574']
#naeDict['otterson'] = ["518", "517", "589", "590"]

In [38]:
comb_rules = {
    'ebu;3;b': 'ebu3b',
    'co;2': 'co2',
    'ap;&;m': 'ap&m',
    'h;2;o': 'h2o'
}

In [39]:
common_map = {'vavb': ['vav', 'b'],
            'lvfd': ['l', 'vfd'],
            'rvfd': ['r', 'vfd'],
            'bsmtsf': ['bsmt', 'sf'],
            'lvlco': ['lvl', 'co'],
            'flsf': ['fl', 'sf'],
            'evfd': ['e', 'vfd'],
            'flah': ['fl', 'ah'],
            'avgzn': ['avg', 'zn'],
            'bsmtah': ['bsmt', 'ah'],
            'rfil': ['r', 'fil'],
            'nef': ['n', 'ef'],
            'rftr': ['r', 'ftr'],
            'ocmin': ['o', 'c', 'min'],
            'ohmin': ['o', 'h', 'min'],
            'ocmnc': ['oc', 'mn', 'c'],
            'mtwdp': ['mtw', 'dp'],
            'cwdp': ['cw', 'dp'],
            'oclg': ['o', 'clg'],
            'chwp': ['chw', 'p'],
            'cmax': ['c', 'max'],
            'cmaxflow': ['c', 'max', 'flow'],
            'cmaxflo': ['c', 'max', 'flo'],
            'satstpt': ['sa', 't', 'stpt'],
            'sathtg': ['sa', 't', 'htg'],
            'hmax': ['h', 'max'],
            'easp': ['ea', 'sp'],
            'esp': ['e', 'sp'],
            'sasp': ['sa', 'sp'],
            'chwsys': ['chw', 'sys'],
            'mtws': ['mtw', 's'],
            'oat': ['oa', 't'],
            'minpos': ['min', 'pos'],
            'ohtg': ['o', 'htg'],
            'occhtgfl': ['occ', 'htg', 'fl'],
            'chws': ['chw', 's'],
            'pmpflwsts': ['pmp', 'flw', 'sts'],
            'saspstpt': ['sa', 'sp', 'stpt'],
            'satclg': ['sa', 't', 'clg'],
            'hthwr': ['h', 't', 'h', 'w', 'r'],
            'occhtg': ['occ', 'htg'],
            'unchmax': ['unch', 'max'],
            'chwr': ['chw', 'r'],
            'bchw': ['b', 'chw'],
            'occhtg': ['occ', 'htg'],
            'lowosa': ['low', 'osa'],
            'mtwsrst': ['mtw', 's', 'rst'],
            'sat': ['sa', 't'],
            'ucmnc': ['uc', 'mn', 'c'],
            'chwdpstpt': ['chw', 'dp', 'stpt'],
            'lfil': ['l', 'fil'],
            'chwdp': ['chw', 'dp'],
            'cwdp': ['cw', 'dp'],
            'boxmode': ['box', 'mode'],
            'supflosp': ['sup', 'flo', 'sp'],
            'commonsp': ['common', 'sp'],
            'dmprpos': ['dmpr', 'pos'],
            'airflow': ['air', 'flow'],
            'mtwr': ['mtw', 'r'],
            'pband': ['p', 'band'],
            'minpossp': ['min', 'pos', 'sp'],
              'rfvfd': ['rf', 'vfd'],
              'deadband': ['dead', 'band'],
              'dehsetpt': ['deh', 'setpt'],
              'lftr': ['l', 'ftr'],
              'occclg': ['occ', 'clg'],
              'bldgstatic': ['bldg', 'static'],
              'weststatic': ['west', 'static'],
              'buildingstatic': ['building', 'static'],
              'hitempalm': ['hi', 'temp', 'alm'],
              'clgminflow': ['clg', 'min', 'flow'],
              'aclg': ['a', 'clg'],
              'ahtg': ['a', 'htg'],
              'sfvfd': ['sf', 'vfd'],
              'rat': ['ra', 't'],
              'econdif': ['econ', 'dif'],
              'occsens': ['occ', 'sens'],
              'omin': ['o', 'min'],
              'clgpid': ['clg', 'pid'],
              'htgpid': ['htg', 'pid'],
              'hihum': ['hi', 'hum'],
              'hitemp': ['hi', 'temp'],
              'shortcyc': ['short', 'cyc'],
              'hwdp': ['hw', 'dp'],
              'ufloor': ['u', 'floor'],
              'ufllr': ['u', 'flr'],
              'svfd': ['s', 'vfd'],
              'supflo': ['sup', 'flo'],
              'suplo': ['sup', 'lo'],
              'supflow': ['sup', 'flow'],
              'inunocc': ['in', 'unocc']
             }

In [40]:


token_map_dict = dict()
token_map_dict['ap_m'] = {
    'easf': ['ea', 'sf'],
    'omin': ['o', 'min'],
    'vavb': ['vav', 'b'],
    'lvfd': ['l', 'vfd'],
    'rvfd': ['r', 'vfd'],
    'bsmtsf': ['bsmt', 'sf'],
    'lvlco': ['lvl', 'co'],
    'flsf': ['fl', 'sf'],
    'evfd': ['e', 'vfd'],
    'flah': ['fl', 'ah'],
    'avgzn': ['avg', 'zn'],
    'bsmtah': ['bsmt', 'ah'],
    'rfil': ['r', 'fil'],
    'nef': ['n', 'ef'],
    'rftr': ['r', 'ftr'],
    'ocmin': ['o', 'c', 'min'],
    'ohmin': ['o', 'h', 'min'],
    'ocmnc': ['oc', 'mn', 'c'],
    'mtwdp': ['mtw', 'dp'],
    'cwdp': ['cw', 'dp'],
    'oclg': ['o', 'clg'],
    'chwp': ['chw', 'p'],
    'cmax': ['c', 'max'],
    'cmaxflow': ['c', 'max', 'flow'],
    'cmaxflo': ['c', 'max', 'flo'],
    'satstpt': ['sa', 't', 'stpt'],
    'sathtg': ['sa', 't', 'htg'],
    'hmax': ['h', 'max'],
    'easp': ['ea', 'sp'],
    'esp': ['e', 'sp'],
    'sasp': ['sa', 'sp'],
    'chwsys': ['chw', 'sys'],
    'mtws': ['mtw', 's'],
    'oat': ['oa', 't'],
    'minpos': ['min', 'pos'],
    'ohtg': ['o', 'htg'],
    'occhtgfl': ['occ', 'htg', 'fl'],
    'chws': ['chw', 's'],
    'pmpflwsts': ['pmp', 'flw', 'sts'],
    'saspstpt': ['sa', 'sp', 'stpt'],
    'satclg': ['sa', 't', 'clg'],
    'hthwr': ['h', 't', 'h', 'w', 'r'],
    'occhtg': ['occ', 'htg'],
    'unchmax': ['unch', 'max'],
    'chwr': ['chw', 'r'],
    'bchw': ['b', 'chw'],
    'occhtg': ['occ', 'htg'],
    'lowosa': ['low', 'osa'],
    'mtwsrst': ['mtws', 'rst'],
    'sat': ['sa', 't'],
    'ucmnc': ['uc', 'mn', 'c'],
    'chwdpstpt': ['chw', 'dp', 'stpt'],
    'lfil': ['l', 'fil'],
    'chwdp': ['chw', 'dp'],
    'cwdp': ['cw', 'dp'],
    'boxmode': ['box', 'mode'],
    'supflosp': ['sup', 'flo', 'sp'],
    'aclg': ['a', 'clg'],
    'aht': ['a', 'htg'],
    'eaf': ['ea', 'f'],
    'bchwr': ['b', 'chw', 'r'],
    'bchws': ['b', 'chw', 's'],
    'fcw': ['f', 'cw'],
    'hthws': ['ht', 'hw', 's']
}
token_map_dict['ebu3b'] = {
    'mtwsclgpid': ['mtw', 's', 'clg', 'pid'],
    'cwsp': ['cw', 'sp'],
    'fschw': ['fs', 'chw'],
    'fcwsp': ['f', 'cw', 'sp'],
    'tempsetf': ['tempset', 'f'],
    'brm': ['b', 'rm'],
    'rmb': ['rm', 'b'],
    'conferenceroom': ['conference', 'room'],
    'clgmaxflo': ['clg', 'max', 'flo'],
    'clgminflo': ['clg', 'min', 'flo'],
    'actclgsp': ['act', 'clg', 'sp'],
    'acthtgsp': ['act', 'htg', 'sp'],
    'commonsp': ['common', 'sp'],
    'htgflo': ['htg', 'flo'],
    'supflosp': ['sup', 'flo', 'sp'],
    'ocmin': ['o', 'c', 'min'],
    'ohmin': ['o', 'h', 'min'],
    'chwdp': ['chw', 'dp'],
    'chngfilt': ['chng', 'filt'],
    'mtwr': ['mtw', 'r'],
    'suplosp': ['sup', 'lo', 'sp'],
    'hihitempalm': ['hi', 'hi', 'temp', 'alm'],
    'mtws': ['mtw', 's'],
    'mtwdp': ['mtw', 'dp'],
    'alrmpres': ['alrm', 'pres'],
    'chws': ['chw', 's'],
    'htgflow': ['htg', 'flow'],
    'chwdpstpt': ['chw', 'dp', 'stpt'],
    'chwr': ['chw', 'r'],
    'occsen': ['occ', 'sen'],
    'dmprpos': ['dmpr', 'pos'],
    'chwp': ['chw', 'p'],
    'sat': ['sa', 't'],
    'lohum': ['lo', 'hum'],
    'cwdp': ['cw', 'dp'],
    'boxmode': ['box', 'mode'],
    'cmaxflo': ['c', 'max', 'flow'],
    'ocmnc': ['oc', 'mn', 'c'],
    'occclg': ['occ', 'clg'],
    'tempalm': ['temp', 'alm'],
    'fcw': ['f', 'cw']
}

token_map_dict['bml'] = {
    'fsd': ['f', 'sd'],
    'stfl': ['st', 'fl'],
    'ndfl': ['nd', 'fl'],
    'unocmnc': ['unoc', 'mn', 'c'],
    'tocc': ['t', 'occ'],
    'dehpb': ['deh', 'pb'],
    'supflow': ['sup', 'flow'],
    'dband': ['d', 'band'],
    'fsds': ['f', 'sd', 's'],
    'elcrm': ['elc', 'rm'],
    'bst': ['b', 'st'],
    'dat': ['da', 't'],
    'handauto': ['hand', 'auto'],
    'rfpid': ['rf', 'pid'],
    'stopdriv': ['stop' ,'vfd'],
    'elcrmt': ['elc', 'rm', 't'],
    'elecrm': ['elec', 'rm'],
    'pipb': ['pi', 'pb'],
    'dhudb': ['dhu', 'db'],
    'econdif': ['econ', 'dif'],
    'aclg': ['a', 'clg'],
    'aht': ['a', 'htg'],
    'dhuit': ['dhu', 'it'],
    'pistpt': ['pi' ,'stpt']
}
token_map_dict['pouya'] = {
    'hicrnt': ['hi', 'crnt'],
    'evav': ['e', 'vav']
}
token_map_dict['ap_m'].update(common_map)
token_map_dict['bml'].update(common_map)
token_map_dict['ebu3b'].update(common_map)
token_map_dict['sme'] = {}
token_map_dict['music'] = {}
token_map_dict['pouya'].update(common_map)


In [41]:
#buildingNameList = ['ebu3b', 'ap_m', 'bml']#sme', 'music']
buildingNameList = ['ebu3b', 'ap_m', 'bml']
notUcsdBuildings = ['ghc']

adder = lambda x,y:x+y

for buildingName in buildingNameList:
    token_map = token_map_dict[buildingName]
    def word_tokenize(word):
        if token_map.get(word):
            return token_map[word]
        else:
            return [word]
    #for tokenType in ['AlphaAndNum', 'JustSeparate']:
    for tokenType in ['JustSeparate']:
        if not buildingName in notUcsdBuildings:
            sensorDF, srcid_list, name_list, jciname_list, desc_list, unit_list, bacnettype_list =\
                toker.parse_sentences(buildingName)
        sentence_dict = dict()
        char_sentence_dict = dict()
        for i, row in enumerate(zip(name_list, jciname_list, desc_list)):
            srcid = srcid_list[i]
            para = list()
            for ele in row:
                raw_tokenized = reduce(adder, map(word_tokenize, ele), [])
                tokenized = []
                begin_idx = 0
                while begin_idx < len(raw_tokenized):
                    combined = ''
                    for delta_idx in range(1, 5):
                        combining = ';'.join(raw_tokenized[begin_idx:begin_idx+delta_idx+1])
                        if combining in comb_rules:
                            combined = comb_rules[combining]
                            break
                    if combined:
                        tokenized.append(combined)
                        begin_idx += delta_idx + 1
                    else:
                        tokenized.append(raw_tokenized[begin_idx])
                        begin_idx += 1
                para.extend(tokenized)
                para.append('\n')
                
            sentence_dict[srcid] = para
            
            char_sentence_dict[srcid] = list(reduce(adder, para, ''))
        with open("metadata/%s_sentence_dict_justseparate.json" % (buildingName), "w") as fp:
            json.dump(sentence_dict, fp, indent=2, separators=(',',':'))
        with open("metadata/%s_sentence_dict.json" % (buildingName), "w") as fp:
            json.dump(sentence_dict, fp, indent=2, separators=(',',':'))
        with open("metadata/%s_char_sentence_dict_justseparate.json" % (buildingName), "w") as fp:
            json.dump(char_sentence_dict, fp, indent=2, separators=(',',':'))
        with open("metadata/%s_char_sentence_dict.json" % (buildingName), "w") as fp:
            json.dump(char_sentence_dict, fp, indent=2, separators=(',',':'))

In [42]:
print("Finished!!!")
sound_file = 'etc/fins_success.wav'
Audio(url=sound_file, autoplay=True)

Finished!!!
